# CLIPエンベディングとGPT-4 Visionを使用したマルチモーダルRAG

マルチモーダルRAGは、従来のテキストベースのRAGに追加のモダリティを統合し、追加のコンテキストを提供してテキストデータをグラウンディングすることで、LLMの質問応答を強化し、理解を向上させます。

[clothing matchmaker cookbook](https://cookbook.openai.com/examples/how_to_combine_gpt4v_with_rag_outfit_assistant)のアプローチを採用し、テキストキャプション化という情報損失を伴うプロセスを回避して画像を直接埋め込み、類似性検索を行うことで、検索精度を向上させています。

CLIPベースの埋め込みを使用することで、特定のデータでのファインチューニングや、未見の画像での更新がさらに可能になります。

この技術は、ユーザーが提供した技術画像を使用して企業のナレッジベースを検索し、関連情報を提供することを通じて実演されています。

# インストール

まず、関連するパッケージをインストールしましょう。

In [ ]:
#installations
%pip install clip
%pip install torch
%pip install pillow
%pip install faiss-cpu
%pip install numpy
%pip install git+https://github.com/openai/CLIP.git
%pip install openai

それでは、必要なパッケージをすべてインポートしましょう。

In [3]:
# model imports
import faiss
import json
import torch
from openai import OpenAI
import torch.nn as nn
from torch.utils.data import DataLoader
import clip
client = OpenAI()

# helper imports
from tqdm import tqdm
import json
import os
import numpy as np
import pickle
from typing import List, Union, Tuple

# visualisation imports
from PIL import Image
import matplotlib.pyplot as plt
import base64

それでは、CLIPモデルを読み込みましょう。

In [4]:
#load model on device. The device you are running inference/training on is either a CPU or GPU if you have.
device = "cpu"
model, preprocess = clip.load("ViT-B/32",device=device)

以下を実行します：
1. 画像埋め込みデータベースを作成する
2. ビジョンモデルへのクエリを設定する
3. セマンティック検索を実行する
4. ユーザークエリを画像に渡す

# 画像埋め込みデータベースの作成

次に、画像のディレクトリから画像埋め込みナレッジベースを作成します。これは、ユーザーがアップロードした画像に関する情報を提供するために検索する技術のナレッジベースとなります。

画像を保存するディレクトリ（JPEG形式）を渡し、それぞれをループして埋め込みを作成します。

また、`description.json`も用意しています。これには、ナレッジベース内のすべての画像に対するエントリが含まれています。このファイルには`'image_path'`と`'description'`の2つのキーがあり、各画像をユーザーの質問に答える際に役立つ画像の説明にマッピングしています。

まず、指定されたディレクトリ内のすべての画像パスを取得する関数を書きましょう。その後、'image_database'というディレクトリからすべてのjpegファイルを取得します。

In [5]:
def get_image_paths(directory: str, number: int = None) -> List[str]:
    image_paths = []
    count = 0
    for filename in os.listdir(directory):
        if filename.endswith('.jpeg'):
            image_paths.append(os.path.join(directory, filename))
            if number is not None and count == number:
                return [image_paths[-1]]
            count += 1
    return image_paths
direc = 'image_database/'
image_paths = get_image_paths(direc)


次に、一連のパスが与えられたときにCLIPモデルから画像埋め込みを取得する関数を書きます。

まず、先ほど取得した前処理関数を使用して画像を前処理します。これにより、CLIPモデルへの入力が適切な形式と次元になるよう、リサイズ、正規化、色チャンネル調整などのいくつかの処理が実行されます。

次に、これらの前処理済み画像をスタックして、ループではなく一度にモデルに渡せるようにします。そして最後に、埋め込みの配列であるモデル出力を返します。

In [6]:
def get_features_from_image_path(image_paths):
  images = [preprocess(Image.open(image_path).convert("RGB")) for image_path in image_paths]
  image_input = torch.tensor(np.stack(images))
  with torch.no_grad():
    image_features = model.encode_image(image_input).float()
  return image_features
image_features = get_features_from_image_path(image_paths)


これで、ベクトルデータベースを作成できるようになりました。

In [7]:
index = faiss.IndexFlatIP(image_features.shape[1])
index.add(image_features)


また、画像と説明のマッピング用のjsonを取り込み、jsonのリストを作成します。さらに、指定した画像を検索するためのヘルパー関数も作成し、その画像の説明を取得できるようにします。

In [11]:
data = []
image_path = 'train1.jpeg'
with open('description.json', 'r') as file:
    for line in file:
        data.append(json.loads(line))
def find_entry(data, key, value):
    for entry in data:
        if entry.get(key) == value:
            return entry
    return None

ユーザーがアップロードした画像の例を表示してみましょう。これは2024年のCESで発表された技術製品です。DELTA Pro Ultra Whole House Battery Generatorです。

In [ ]:
im = Image.open(image_path)
plt.imshow(im)
plt.show()

![Delta Pro](../images/train1.jpeg)

# ビジョンモデルのクエリ

それでは、GPT-4 Vision（この技術を以前に見たことがないはず）が、これを何とラベル付けするかを見てみましょう。

まず、画像をbase64形式でエンコードする関数を作成する必要があります。これは、ビジョンモデルに渡す形式だからです。次に、画像入力でLLMにクエリを送信できるように、汎用的な`image_query`関数を作成します。

In [12]:
def encode_image(image_path):
    with open(image_path, 'rb') as image_file:
        encoded_image = base64.b64encode(image_file.read())
        return encoded_image.decode('utf-8')

def image_query(query, image_path):
    response = client.chat.completions.create(
        model='gpt-4-vision-preview',
        messages=[
            {
            "role": "user",
            "content": [
                {
                "type": "text",
                "text": query,
                },
                {
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/jpeg;base64,{encode_image(image_path)}",
                },
                }
            ],
            }
        ],
        max_tokens=300,
    )
    # Extract relevant features from the response
    return response.choices[0].message.content
image_query('Write a short label of what is show in this image?', image_path)

'Autonomous Delivery Robot'

ご覧のとおり、AIは訓練されたデータの情報から最善を尽くそうとしますが、訓練データで類似したものを見たことがないため、間違いを犯してしまいます。これは、曖昧な画像であるため、推定や推論が困難になるからです。

# セマンティック検索の実行

それでは、類似度検索を実行して、ナレッジベース内で最も類似している2つの画像を見つけてみましょう。これは、ユーザーが入力した`image_path`の埋め込みを取得し、データベース内の類似画像のインデックスと距離を取得することで行います。距離は類似度の代理指標となり、距離が小さいほどより類似していることを意味します。その後、距離に基づいて降順でソートします。

In [13]:
image_search_embedding = get_features_from_image_path([image_path])
distances, indices = index.search(image_search_embedding.reshape(1, -1), 2) #2 signifies the number of topmost similar images to bring back
distances = distances[0]
indices = indices[0]
indices_distances = list(zip(indices, distances))
indices_distances.sort(key=lambda x: x[1], reverse=True)

インデックスが必要です。これを使用して`image_directory`を検索し、インデックスの位置にある画像を選択してRAG用のビジョンモデルに入力するためです。

そして、何が返されたかを見てみましょう（類似度の順に表示しています）：

In [ ]:
#display similar images
for idx, distance in indices_distances:
    print(idx)
    path = get_image_paths(direc, idx)[0]
    im = Image.open(path)
    plt.imshow(im)
    plt.show()

![Delta Pro2](../images/train2.jpeg)

![Delta Pro3](../images/train17.jpeg)

ここでは、DELTA Pro Ultra Whole House Battery Generatorが含まれている2つの画像が返されていることがわかります。画像の1つには気を散らす可能性のある背景も含まれていますが、適切な画像を見つけることができています。

# 最も類似した画像をクエリするユーザー

最も類似した画像について、その画像と説明をユーザーのクエリと共にgpt-vに渡して、購入した可能性のある技術について問い合わせできるようにします。これがビジョンモデルの力が発揮される部分で、モデルが明示的に訓練されていない一般的なクエリを投げかけても、高い精度で応答してくれます。

以下の例では、問題となっているアイテムの容量について問い合わせを行います。

In [14]:
similar_path = get_image_paths(direc, indices_distances[0][0])[0]
element = find_entry(data, 'image_path', similar_path)

user_query = 'What is the capacity of this item?'
prompt = f"""
Below is a user query, I want you to answer the query using the description and image provided.

user query:
{user_query}

description:
{element['description']}
"""
image_query(prompt, similar_path)

'The portable home battery DELTA Pro has a base capacity of 3.6kWh. This capacity can be expanded up to 25kWh with additional batteries. The image showcases the DELTA Pro, which has an impressive 3600W power capacity for AC output as well.'

そして、質問に答えることができていることがわかります。これは、画像を直接マッチングし、そこから関連する説明をコンテキストとして収集することによってのみ可能でした。

# 結論

このノートブックでは、CLIPモデルの使用方法、CLIPモデルを使用した画像埋め込みデータベースの作成例、セマンティック検索の実行、そして最終的にユーザークエリに対して質問に答える方法について説明しました。

この使用パターンの応用は多くの異なるアプリケーションドメインに広がっており、この技術をさらに向上させるために容易に改良することができます。例えば、CLIPをファインチューニングしたり、RAGと同様に検索プロセスを改善したり、GPT-Vをプロンプトエンジニアリングしたりすることができます。